In [22]:
import pandas as pd

In [23]:
df = pd.read_csv("../data/d10800_BS1_JoinedTrans.csv")
df.head()

,src_file,row_id,hg38_ASF1A,hg38_CCT7,hg38_EPOR,hg38_NECAP2,hg38_LRRC8A,hg38_NDUFS8,hg38_MRPL22,hg38_HIST1H2BC,...,hg38_TBC1D10C,hg38_TMEM128,hg38_ASPSCR1,hg38_CDC20,hg38_DNAJB9,hg38_OSTM1,hg38_LCNL1,hg38_PHB,hg38_LRRC29,hg38_ZFP36
0,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,13,0,0,0,3,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,19,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,2,0,3
3,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,21,0,0,0,0,0,3,0,0,...,0,0,0,0,3,0,0,3,0,0
4,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,26,0,0,0,0,0,3,0,0,...,3,0,0,0,0,0,0,0,0,0


In [24]:
X = df.drop(columns=['src_file', 'row_id'])

In [25]:
X.shape

(13183, 10800)

In [33]:
y = pd.array(df['src_file'])

In [27]:
from sklearn.feature_selection import VarianceThreshold

In [28]:
selector = VarianceThreshold()
selector.fit(X)
X_selected = X[X.columns[selector.get_support(indices=True)]]
X_selected.head()

,hg38_ASF1A,hg38_CCT7,hg38_EPOR,hg38_NECAP2,hg38_LRRC8A,hg38_NDUFS8,hg38_MRPL22,hg38_HIST1H2BC,hg38_LY96,hg38_UBE2E2-AS1,...,hg38_TBC1D10C,hg38_TMEM128,hg38_ASPSCR1,hg38_CDC20,hg38_DNAJB9,hg38_OSTM1,hg38_LCNL1,hg38_PHB,hg38_LRRC29,hg38_ZFP36
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,3,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,2,0,3
3,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,3,0,0,3,0,0
4,0,0,0,0,0,3,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0


In [34]:
groups = []
subgroups = []
for s in y:
    src_file = s.split('_')
    group = src_file[2].replace('C','')
    groups.append(group)
    subgroup = src_file[-2].split('-')[0]
    if subgroup == 'OK':
        subgroup = group
    subgroups.append(subgroup)
groups = pd.array(groups)
subgroups = pd.array(subgroups)
y = pd.DataFrame({'Group':groups, 'Subgroups':subgroups})
y.head()

,Group,Subgroups
0,B,Bmemory
1,B,Bmemory
2,B,Bmemory
3,B,Bmemory
4,B,Bmemory


In [35]:
y = pd.DataFrame({'Group':groups, 'Subgroups':subgroups})
y.head()

,Group,Subgroups
0,B,Bmemory
1,B,Bmemory
2,B,Bmemory
3,B,Bmemory
4,B,Bmemory


In [36]:
y['Subgroups'].unique()

<StringArray>
[      'Bmemory',        'Bnaive',           'ALL',        'CD14+M',
        'CD16+M',         'aTreg', 'NonClassicalT',          'nonT',
         'rTreg',     'T4eff.mem',       'T4naive',     'T8eff.mem',
       'T8naive']
Length: 13, dtype: string

In [31]:
from joblib import dump
dump((X_selected, y), 'preprocessed_data_BS1.joblib')

['preprocessed_data_BS1.joblib']

In [12]:
def load_data(data, columns):
    df = pd.read_csv(data)
    X = df[columns]
    y = pd.array(df['src_file'])
    return X, y

In [39]:
X, y = load_data(data="../data/d10800_BS2_JoinedTrans.csv", columns=X_selected.columns)
groups = []
subgroups = []
for s in y:
    src_file = s.split('_')
    group = src_file[2].replace('C','')
    groups.append(group)
    subgroup = src_file[6]
    if subgroup == '8T':
        subgroup = 'T8'
    subgroups.append(subgroup)
groups = pd.array(groups)
subgroups = pd.array(subgroups)
y = pd.DataFrame({'Group':groups, 'Subgroups':subgroups})
y.head()

,Group,Subgroups
0,B,B
1,B,B
2,B,B
3,B,B
4,B,B


In [40]:
y['Subgroups'].unique()

<StringArray>
['B', 'DC', 'pDC', 'V2', 'V3', 'NK', 'T4', 'T8']
Length: 8, dtype: string

In [41]:
dump((X, y), 'preprocessed_data_BS2.joblib')

['preprocessed_data_BS2.joblib']

In [42]:
X, y = load_data(data="../data/d10800_10x_JoinedTrans.csv", columns=X_selected.columns)
groups = []
subgroups = []
for s in y:
    src_file = s.split('_')
    group = src_file[2].replace('C','')
    groups.append(group)
    subgroup = src_file[4].split('-')[-2]
    if subgroup == 'OK':
        subgroup = group
    subgroups.append(subgroup)
groups = pd.array(groups)
subgroups = pd.array(subgroups)
y = pd.DataFrame({'Group':groups, 'Subgroups':subgroups})
y.head()

,Group,Subgroups
0,B,B
1,B,B
2,B,B
3,B,B
4,B,B


In [43]:
y['Subgroups'].unique()

<StringArray>
['B', 'M', 'NK', 'T4h', 'T4mem', 'T4naive', 'T4reg', 'T8', 'T8naive']
Length: 9, dtype: string

In [16]:
dump((X, y), 'preprocessed_data_10x.joblib')

['preprocessed_data_10x.joblib']

In [44]:
X, y = load_data(data="../data/d10800_GEO_JoinedTrans.csv", columns=X_selected.columns)
groups = []
subgroups = []
for s in y:
    src_file = s.split('_')
    group = src_file[2].replace('C','')
    groups.append(group)
    subgroup = group
    subgroups.append(subgroup)
groups = pd.array(groups)
subgroups = pd.array(subgroups)
y = pd.DataFrame({'Group':groups, 'Subgroups':subgroups})
y.head()

,Group,Subgroups
0,B,B
1,B,B
2,B,B
3,B,B
4,B,B


In [45]:
y['Subgroups'].unique()

<StringArray>
['B', 'M', 'NK', 'T']
Length: 4, dtype: string

In [20]:
dump((X, y), 'preprocessed_data_GEO.joblib')

['preprocessed_data_GEO.joblib']